# Writing Functions in Python

## Docstring

In [1]:
def count_letter(content, letter):
  """Count the number of times `letter` appears in `content`.

  Args:
    content (str): The string to search.
    letter (str): The letter to search for.

  Returns:
    int

  # Add a section detailing what errors might be raised
  Raises:
    ValueError: If `letter` is not a one-character string.
  """
  if (not isinstance(letter, str)) or len(letter) != 1:
    raise ValueError('`letter` must be a single character string.')
  return len([char for char in content if char == letter])

In [3]:
import inspect

#printing docstring
print(inspect.getdoc(count_letter))

Count the number of times `letter` appears in `content`.

Args:
  content (str): The string to search.
  letter (str): The letter to search for.

Returns:
  int

# Add a section detailing what errors might be raised
Raises:
  ValueError: If `letter` is not a one-character string.


## Context Manager

	with <context-manager> (<args>) as <variable_name>:
    	#Run code
        #Code is running "inside the context"

<varaible_name> returns the value

## How to create a context manager
	@contextlib.contextmanager
    def my_context():
    	#Add any set up code you need
        yield
        #Add any teardown code you need
        
1. Define a function.
2. (optional) Add any set up code your context needs.
3. Use the "yield" keyword.
4. (optional) Add any teardown code your context needs.
5. Add the 'contextlib.contextmanager' decorator.

In [10]:
#creating a context manager
from contextlib import contextmanager
@contextmanager
def my_context():
    print("Hello") #setup code
    yield 42
    print("Goodbye") #tear down code

In [7]:
#the yield keyword
with my_context() as foo: #the yield is passed as alias name "foo"
    print(foo)

Hello
42
Goodbye


**Note:** 
- First setup code is run
- Yield statement returns value which is passed to context manager as alias name which was then printed
- Then tear down code is run

## Nested contexts
	def copy(src,des):
    	"""Copy the contents of one file to another.
        
        Args:
        	src(string):File name of the file to be copied
            des(string):Where to write the new file
        """
        #Open both files
        with open(src) as src_file:
        	with open(des,"w") as des_file:
            #Read and write each time, one at a time
            for line in src_file:
            	des_file.write(line)

## Handling errors

	try:
    	#code that might raise an error
    except:
    	#do something about the error
    finally:
    	#this code runs no matter what

**Scraping the NASDAQ**

Training deep neural nets is expensive! You might as well invest in NVIDIA stock since you're spending so much on GPUs. To pick the best time to invest, you are going to collect and analyze some data on how their stock is doing. The context manager stock('NVDA') will connect to the NASDAQ and return an object that you can use to get the latest price by calling its .price() method.

You want to connect to stock('NVDA') and record 10 timesteps of price data by writing it to the file NVDA.txt.

You will notice the use of an underscore when iterating over the for loop. If this is confusing to you, don't worry. It could easily be replaced with i, if we planned to do something with it, like use it as an index. Since we won't be using it, we can use a dummy operator, _, which doesn't use any additional memory.

	# Use the "stock('NVDA')" context manager
    # and assign the result to the variable "nvda"
    with stock('NVDA') as nvda:
      # Open "NVDA.txt" for writing as f_out
      with open ('NVDA.txt',"w") as f_out:
        for _ in range(10):
          value = nvda.price()
          print('Logging ${:.2f} for NVDA'.format(value))
          f_out.write('{:.2f}\n'.format(value))

# Functions are objects

In [5]:
#list and dictionaries of functions 

list_of_functions=[open,print]
list_of_functions[1]("abcd") #on the list 1 index has print function so it prints 

dict_of_functions={
    "func1" : open,
    "func2" : print
}
dict_of_functions["func2"]("efgh") #on dict "func2" has print function so it prints

abcd
efgh


### Functions as arguments

In [1]:
#define a function that checks if the any function contains docstring or not
#takes function as an argument
def has_docstring(function):
    """Check to see of function 'function' has a doctring
    
    Args:
        function(callable): A function
        
    Returns:
        bool
    """
    return function.__doc__ is not None


In [2]:
#define a function with no docstring
def no():
    return 42

In [3]:
#define a function with docstring
def yes():
    """Returns the value 42"""
    return 42

In [4]:
has_docstring(no) # "no" function as an argument 

False

In [5]:
has_docstring(yes) #"yes" function as an argument

True

### Return function as an argument 

In [7]:
#define a function that takes in a function name "func_name"
def create_math_function(func_name):
  if func_name == 'add':
    #Define the add() function
    def add(a, b):
      return a + b
    return add #return add function if func_name == "add"

  elif func_name == 'subtract':
    # Define the subtract() function
    def subtract(a,b):
      return a - b
    return subtract #return subtract function if func_name == "subtract"

  else:
    print("I don't know that one")
    
add = create_math_function('add')
print('5 + 2 = {}'.format(add(5, 2)))

subtract = create_math_function('subtract')
print('5 - 2 = {}'.format(subtract(5, 2)))

5 + 2 = 7
5 - 2 = 3


### Scope 

![image-2](image-2.png)

- First checks the local scope which means within ones function
- Second checks the non-local scope which means within the parent function
- Third checks the global scope i.e outside ones function and parent function
- Finally checks the built-in scope

In [4]:
import random

def wait_until_done():
  def check_is_done():
    # Add a keyword so that wait_until_done() 
    # doesn't run forever
    global done #manipulates global variable
    if random.random() < 0.1:
      done = True
      
  while not done:
    check_is_done()

done = False #global variable "done"
wait_until_done()

print('Work done? {}'.format(done))

Work done? True


### Closure: Nonlocal variables attached to a returned function

In [8]:
def parent(arg1,arg2):
    value = 21
    my_dict={"chocolate":"yummy"}
    
    def child():
        print(value*2)
        print(my_dict["chocolate"])
        print(arg1+arg2)
    
    return child

new_function = parent(3,4)

#print the closure values that new_function which stores child() function still holds
print([cell.cell_contents for cell in new_function.__closure__])

[3, 4, {'chocolate': 'yummy'}, 21]


In [9]:
#calling new_function which stores child() function
new_function()

42
yummy
7


In [10]:
def return_a_func(arg1, arg2):
  def new_func():
    print('arg1 was {}'.format(arg1))
    print('arg2 was {}'.format(arg2))
  return new_func
    
my_func = return_a_func(2, 17)

print(my_func.__closure__ is not None)
print(len(my_func.__closure__) == 2)

# Get the values of the variables in the closure
closure_values = [
  my_func.__closure__[i].cell_contents for i in range(2)
]
print(closure_values == [2, 17])

True
True
True


# Decorators

![image-3](image-3.png)

Can modify the behaviour of function, inputs and outputs

**Note** Decorators are the functions that takes in the function and modify them

In [1]:
#creating decorator that takes in a function
def double_args(func):
    def wrapper(a,b):
        """calls the function "func" that double_args takes, 
        multiply argument by 2 before passing into the func"""
        return func(a*2,b*2) 
    return wrapper #return wrapper function

def multiply(a,b):
    return a*b

#call double_args and pass multiply() function which returns wrapper function
multiply = double_args(multiply) 

#call wrapper function through variable "multiply"
multiply(1,5)

20

In [2]:
#same code as above using decorator syntax 

def double_args(func):
    def wrapper(a,b):
        return func(a*2,b*2)
    return wrapper 

@double_args
def multiply(a,b):
    return a*b

multiply(1,5)

20

## Timer decorator

In [26]:
import time

def timer(func):
    """Decorator that prints how long a function took to run"""
    #Define the wrapper function to return
    def wrapper(*args,**kwargs):
        #when wrapper() is called, get the current time
        t_start = time.time()
        #call the decorated function and store the result
        result = func(*args,**kwargs)
        #get the total time it took to run and print it
        t_total = time.time() - t_start
        print(func.__name__+" took "+str(t_total)+"s")
        
        return result
    return wrapper


In [12]:
@timer
def sleep_n_seconds(n):
    time.sleep(n)

In [13]:
sleep_n_seconds(5)

sleep_n_seconds took 5.005045413970947s


## Creating decorators that returns type 

In [1]:
def print_return_type(func):
  # Define wrapper(), the decorated function
  def wrapper(*args, **kwargs):
    # Call the function being decorated
    result = func(*args, **kwargs)
    print('{}() returned type {}'.format(
      func.__name__, type(result)
    ))
    return result
  # Return the decorated function
  return wrapper
  
@print_return_type
def foo(value):
  return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


## Counter

In [3]:
def counter(func):
    def wrapper(*args,**kwargs):
        wrapper.count += 1
        return func()
    wrapper.count = 0
    return wrapper

@counter
def foo():
    print("calling foo..")

foo()
foo()
#this foo = counter(foo) so, foo stores wrapper() function 
#foo() calles the wrapper function and hence increaing wrapper.count references foo.count

print("foo is called {} times".format(foo.count))

calling foo..
calling foo..
foo is called 2 times


## Decorators and metadata

The metadata of the decorating function is obscured because its actually referencing the nested function that was returned by the decorator. 

To preserve the metadata of decorating function:

	from functools import wraps
    
    @wraps(func) <-- decorator above the nested function of the main decorator function
    
**It will modify the wrapper's metadata to look like the decorating function**

In [1]:
from functools import wraps

def add_hello(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func)
  def wrapper(*args, **kwargs):
    """Print 'hello' and then call the decorated function."""
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
Adds two numbers and prints the sum


## Decorators that take arguments

In [1]:
def run_n_times(n):
    """Define and return decorator"""
    def decorator(func):
        def wrapper(*args,**kwargs):
            for i in range(n):
                func(*args,**kwargs)
        return wrapper
    return decorator

@run_n_times(3) #decorator with an argument
def sum(a,b):
    print(a+b)
    
sum(5,9)

14
14
14


In [2]:
@run_n_times(5)
def print_hello():
    print("Hello!")
    
print_hello()

Hello!
Hello!
Hello!
Hello!
Hello!


## HTML Generator using Decorator

In [12]:
def html(open_tag,close_tag):
    """decorator with argument that returns the decorator"""
    def decorator(func):
        """decorator that manipulates the given function and returns wrapper"""
        def wrapper(*args,**kwargs):
            """takes in the argument of the function called, 
            calles the function and returns the html code"""
            msg = func(*args,**kwargs)
            return("{}{}{}".format(open_tag,msg,close_tag))
        return wrapper
    return decorator

In [14]:
#create html code for bold 
@html("<b>","</b>")
def msg(name):
    return("Hello {}".format(name))

msg("Jinu Nyachhyon")

'<b>Hello Jinu Nyachhyon</b>'

In [15]:
#create html code for italic
@html("<i>","</i>")
def ilove(statement):
    return("{} YES!".format(statement))

ilove("I love pineapple on pizza")

'<i>I love pineapple on pizza YES!</i>'

In [24]:
#create html code for div
@html("<div>","</div>")
def combo(name,statement):
    return("   {}  {}  ".format(msg(name),ilove(statement)))

combo("Jinu Nyachhyon","I love pineapple")

'<div>   <b>Hello Jinu Nyachhyon</b>  <i>I love pineapple YES!</i>  </div>'

## Timeout() : a real world example

In [2]:
import signal 

def timeout(n_seconds):
    """returns timeout error after n_seconds"""
    def decorator(func):
        def wrapper(*args,**kwargs):
            signal.alarm(n_seconds) #setup alarm for n_seconds
            
            try:
                #run the function which eventually starts the alarm and if alarm goes off,
                #timeout error is displayed
                return func(*args,**kwargs) 
            finally:
                #no matter what cancel the alarm
                signal.alarm(0)
                
        return wrapper
    return decorator
        

In [3]:
import time

@timeout(5)
def sleep_for_2():
    time.sleep(2)
    print("Foo!!")
    
sleep_for_2()

Foo!!


In [ ]:
@timeout(2)
def sleep_for_5():
    time.sleep(5)
    print("Foo!!")
    
sleep_for_5() #gets timeout error / run cancelled

## Tag your functions

Tagging something means that you have given that thing one or more strings that act as labels. For instance, we often tag emails or photos so that we can search for them later. You've decided to write a decorator that will let you tag your functions with an arbitrary list of tags. You could use these tags for many things:

- Adding information about who has worked on the function, so a user can look up who to ask if they run into trouble using it.
- Labeling functions as "experimental" so that users know that the inputs and outputs might change in the future.
- Marking any functions that you plan to remove in a future version of the code.
- Etc.

In [7]:
from functools import wraps

def tag(*tags):
  # Define a new decorator, named "decorator", to return
  def decorator(func):
    # Ensure the decorated function keeps its metadata
    @wraps(func)
    def wrapper(*args, **kwargs):
      # Call the function being decorated and return the result
      return func(*args, **kwargs)
    wrapper.tags = tags
    return wrapper
  # Return the new decorator
  return decorator

@tag('test', 'this is a tag')
def foo():
  pass

print(foo.tags)

('test', 'this is a tag')
